# Preparing

In [1]:
import pandas as pd
submission = pd.read_parquet('../test.parquet').drop(columns = ['rating', 'sale', 'shop_title'])

In [2]:
submission

,product_id,shop_id,text_fields
1,1997646,22758,"{""title"": ""Светодиодная лента Smart led Strip ..."
2,927375,17729,"{""title"": ""Стекло ПЛЕНКА керамик матовое Honor..."
3,1921513,54327,"{""title"": ""Проводные наушники с микрофоном jac..."
4,1668662,15000,"{""title"": ""Декоративная табличка \""Правила кух..."
5,1467778,39600,"{""title"": ""Подставка под ложку керамическая, п..."
...,...,...,...
24987,1914264,8598,"{""title"": ""Жесткий диск внутренний SSD KingDia..."
24988,1310569,27474,"{""title"": ""Браслет оберег на руку/красная нить..."
24989,978095,23395,"{""title"": ""Кабошон бантик в упаковке 2 шт"", ""d..."
24992,797547,16764,"{""title"": ""Полка для ванной угловая, 20,5 х 20..."


In [3]:
import json
def parse_desc(text):
    titles = []
    desc = []
    for i, doc in enumerate(text):
        titles.append(json.loads(doc)['title'])
        desc.append(json.loads(doc)['description'])
    return titles, desc

In [4]:
submission['title'], submission['description'] = parse_desc(list(submission['text_fields']))
submission.drop(columns=['text_fields'], inplace = True)

In [5]:
def clean_description(corpus): # функция для очистки значений как <что-то>
    
    to_output = True
    out_str = ''
    output = []
    
    for text in corpus:
        for ch in text:
            
            if ch == '<':
                to_output = False
                
            if ch == '>':
                to_output = True
                continue
                
            if to_output:
                out_str += ch
                
        output.append(out_str)
        out_str = ''
    
    return output
submission['description'] = clean_description(list(submission['description']))

In [6]:
submission

,product_id,shop_id,title,description
1,1997646,22758,"Светодиодная лента Smart led Strip Light, с пу...","Светодиодная лента LED, 5 м, RGB (Цветная) вла..."
2,927375,17729,Стекло ПЛЕНКА керамик матовое Honor 50 lite 10...,Защитное матовое керамическое стекло эффективн...
3,1921513,54327,"Проводные наушники с микрофоном jack 3.5, IOS,...",Наушники проводные с микрофоном отличное качес...
4,1668662,15000,"Декоративная табличка ""Правила кухни"", подстав...","Декоративная табличка ""Правила кухни"" создаст ..."
5,1467778,39600,"Подставка под ложку керамическая, подложка ""Кл...",Подложка керамическая с рисунком в подарочной ...
...,...,...,...,...
24987,1914264,8598,Жесткий диск внутренний SSD KingDian 2.5 Inch ...,"Важно! Почему на SSD накопителе меньше места, ..."
24988,1310569,27474,Браслет оберег на руку/красная нить от сглаза/...,"Красная нить - оберег, обладающий большой сило..."
24989,978095,23395,Кабошон бантик в упаковке 2 шт,"Кабошон бантик, желтого цвета. Используется дл..."
24992,797547,16764,"Полка для ванной угловая, 20,5 х 20,5 х 6,5 см...","Полка для ванной угловая, 20,5×20,5×6,5 см, цв..."


# Lemmatize

In [7]:
import nltk
from nltk import word_tokenize
nltk.download("punkt")
nltk.download('stopwords')
from nltk.corpus import stopwords 
from nltk.stem import SnowballStemmer
from sklearn.pipeline import Pipeline
import string
import numpy as np
import pymorphy2

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\vnvof\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vnvof\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
ru_sw = stopwords.words("russian") # ru stop words
snowball = SnowballStemmer(language = "russian")# stemming
morph = pymorphy2.MorphAnalyzer()

def token_all_proccesing(df: str, remove_stop_words: bool = True): # func
    output = ''
    tokens = word_tokenize(df, language="russian") # tokenize
    tokens = [i for i in tokens if i not in string.punctuation] # tokenize w/o punctuation
    if remove_stop_words:
        tokens = [i for i in tokens if i not in ru_sw]# remove ru stopword
    #tokens = [snowball.stem(i) for i in tokens] # snowball stemming
    tokens = [morph.parse(i)[0].normal_form for i in tokens] # lemmatization
    
    for i in tokens:
        output += i + ' '
    
    return output

In [9]:
submission.description = submission.description.apply(func = lambda x: token_all_proccesing((x)))

In [10]:
def to_lower(text):
    text = text.split()
    output = ''
    for i in text:
        output += i.lower() + ' '
    return output

In [11]:
submission.title = submission.title.apply(func = lambda x: to_lower(x))

In [12]:
submission

,product_id,shop_id,title,description
1,1997646,22758,"светодиодная лента smart led strip light, с пу...",светодиодный лента led 5 м rgb цветной влагост...
2,927375,17729,стекло пленка керамик матовое honor 50 lite 10...,защитный матовый керамический стекло эффективн...
3,1921513,54327,"проводные наушники с микрофоном jack 3.5, ios,...",наушник проводной микрофон отличный качество з...
4,1668662,15000,"декоративная табличка ""правила кухни"", подстав...",декоративный табличка `` правило кухня '' созд...
5,1467778,39600,"подставка под ложку керамическая, подложка ""кл...",подложка керамический рисунок подарочный упако...
...,...,...,...,...
24987,1914264,8598,жесткий диск внутренний ssd kingdian 2.5 inch ...,важно почему ssd накопитель маленький место за...
24988,1310569,27474,браслет оберег на руку/красная нить от сглаза/...,красный нить оберег обладать большой сила совм...
24989,978095,23395,кабошон бантик в упаковке 2 шт,кабошон бантик жёлтый цвет использоваться разл...
24992,797547,16764,"полка для ванной угловая, 20,5 х 20,5 х 6,5 см...","полка ванная угловой 20,5×20,5×6,5 смотреть цв..."


# Big Category Prediction

In [13]:
from catboost import CatBoostClassifier
main_cat_model = CatBoostClassifier()
main_cat_model.load_model('models/model_catboost_big_cat')

C:\Users\vnvof\anaconda3\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [14]:
submission['category'] = main_cat_model.predict(submission.drop(columns=['product_id']))

# Predicting Sub

In [15]:
submission_cat_0 = submission[submission['category'] == 0].drop(columns=['category'])
submission_cat_1 = submission[submission['category'] == 1].drop(columns=['category'])
submission_cat_2 = submission[submission['category'] == 2].drop(columns=['category'])
submission_cat_3 = submission[submission['category'] == 3].drop(columns=['category'])
submission_cat_4 = submission[submission['category'] == 4].drop(columns=['category'])

In [16]:
cat_0_model_sub = CatBoostClassifier().load_model('models/model_catboost_cat_0_sub')
cat_1_model_sub = CatBoostClassifier().load_model('models/model_catboost_cat_1_sub')
cat_2_model_sub = CatBoostClassifier().load_model('models/model_catboost_cat_2_sub')
cat_3_model_sub = CatBoostClassifier().load_model('models/model_catboost_cat_3_sub')
cat_4_model_sub = CatBoostClassifier().load_model('models/model_catboost_cat_4_sub')

In [19]:
submission_cat_0['category'] = cat_0_model_sub.predict(submission_cat_0.drop(columns=['product_id']))
submission_cat_1['category'] = cat_1_model_sub.predict(submission_cat_1.drop(columns=['product_id']))
submission_cat_2['category'] = cat_2_model_sub.predict(submission_cat_2.drop(columns=['product_id']))
submission_cat_3['category'] = cat_3_model_sub.predict(submission_cat_3.drop(columns=['product_id']))
submission_cat_4['category'] = cat_4_model_sub.predict(submission_cat_4.drop(columns=['product_id']))

# Final Prediction

In [20]:
cat_0_model = CatBoostClassifier().load_model('models/model_catboost_cat_0')
cat_1_model = CatBoostClassifier().load_model('models/model_catboost_cat_1')
cat_2_model = CatBoostClassifier().load_model('models/model_catboost_cat_2')
cat_3_model = CatBoostClassifier().load_model('models/model_catboost_cat_3')
cat_4_model = CatBoostClassifier().load_model('models/model_catboost_cat_4')

In [23]:
def swap_columns(df, col1, col2):
    col_list = list(df.columns)
    x, y = col_list.index(col1), col_list.index(col2)
    col_list[y], col_list[x] = col_list[x], col_list[y]
    df = df[col_list]
    return df

In [24]:
submission_cat_0 = swap_columns(swap_columns(submission_cat_0, 'title', 'category'), 'title', 'description')
submission_cat_1 = swap_columns(swap_columns(submission_cat_1, 'title', 'category'), 'title', 'description')
submission_cat_2 = swap_columns(swap_columns(submission_cat_2, 'title', 'category'), 'title', 'description')
submission_cat_3 = swap_columns(swap_columns(submission_cat_3, 'title', 'category'), 'title', 'description')
submission_cat_4 = swap_columns(swap_columns(submission_cat_4, 'title', 'category'), 'title', 'description')

In [28]:
predicted_0 = cat_0_model.predict(submission_cat_0.drop(columns=['product_id']))
predicted_1 = cat_1_model.predict(submission_cat_1.drop(columns=['product_id']))
predicted_2 = cat_2_model.predict(submission_cat_2.drop(columns=['product_id']))
predicted_3 = cat_3_model.predict(submission_cat_3.drop(columns=['product_id']))
predicted_4 = cat_4_model.predict(submission_cat_4.drop(columns=['product_id']))

In [29]:
import numpy as np
predicted = np.append(predicted_0, predicted_1)
predicted = np.append(predicted, predicted_2)
predicted = np.append(predicted, predicted_3)
predicted = np.append(predicted, predicted_4)

In [32]:
final = submission_cat_0.append(submission_cat_1).append(submission_cat_2).append(submission_cat_3).append(submission_cat_4)

C:\Users\vnvof\AppData\Local\Temp\ipykernel_13660\698243201.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final = submission_cat_0.append(submission_cat_1).append(submission_cat_2).append(submission_cat_3).append(submission_cat_4)
C:\Users\vnvof\AppData\Local\Temp\ipykernel_13660\698243201.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final = submission_cat_0.append(submission_cat_1).append(submission_cat_2).append(submission_cat_3).append(submission_cat_4)
C:\Users\vnvof\AppData\Local\Temp\ipykernel_13660\698243201.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final = submission_cat_0.append(submission_cat_1).append(submission_cat_2).append(submission_cat_3).append(submission_cat_4)
C:\Users\vnvof\AppData

In [34]:
predicted

array([11714, 14922,  2803, ..., 14119, 13894, 14614], dtype=int64)

In [37]:
submission

,product_id,shop_id,title,description,category
1,1997646,22758,"светодиодная лента smart led strip light, с пу...",светодиодный лента led 5 м rgb цветной влагост...,0
2,927375,17729,стекло пленка керамик матовое honor 50 lite 10...,защитный матовый керамический стекло эффективн...,0
3,1921513,54327,"проводные наушники с микрофоном jack 3.5, ios,...",наушник проводной микрофон отличный качество з...,0
4,1668662,15000,"декоративная табличка ""правила кухни"", подстав...",декоративный табличка `` правило кухня '' созд...,3
5,1467778,39600,"подставка под ложку керамическая, подложка ""кл...",подложка керамический рисунок подарочный упако...,3
...,...,...,...,...,...
24987,1914264,8598,жесткий диск внутренний ssd kingdian 2.5 inch ...,важно почему ssd накопитель маленький место за...,0
24988,1310569,27474,браслет оберег на руку/красная нить от сглаза/...,красный нить оберег обладать большой сила совм...,2
24989,978095,23395,кабошон бантик в упаковке 2 шт,кабошон бантик жёлтый цвет использоваться разл...,2
24992,797547,16764,"полка для ванной угловая, 20,5 х 20,5 х 6,5 см...","полка ванная угловой 20,5×20,5×6,5 смотреть цв...",3


In [35]:
sample_submission = pd.DataFrame()
sample_submission['product_id'] = final.product_id
sample_submission['predicted_category_id'] = predicted
sample_submission.to_parquet('result.parquet')